In [1]:
%load_ext dotenv
%dotenv

In [9]:
import warnings

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_upstage import ChatUpstage

warnings.filterwarnings("ignore")


# langchain, 1. llm define, 2. prompt define, 3. chain, 4. chain.invoke

# 1. define your favorate llm, solar
llm = ChatUpstage()

# 2. define chat prompt
history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

history = [
    HumanMessage("What is the capital of France?"),
    AIMessage("It's Paris!!"),
]
latest_query = "What about Korea?"

# 3. define chain
chain = history_prompt | llm | StrOutputParser()

# 4. invoke the chain
print(chain.invoke({"history": history, "input": latest_query}))



###########################################


# # More general chat
# rag_with_history_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant."),
#         MessagesPlaceholder(variable_name="history"),
#         ("human", "{input}"),
#     ]
# )



# history = [
#     HumanMessage("What is the capital of France?"),
#     AIMessage("It's Paris!!"),
# ]

# chain = rag_with_history_prompt | llm | StrOutputParser()
# chain_result = chain.invoke({"history": history, "input": "What about Korea?"})
# print(chain_result)

Oh, in Korea, it's Seoul!!
